In [1]:
%bash
git clone http://www.github.com/GoogleCloudPlatform/training-data-analyst

Cloning into 'training-data-analyst'...


In [18]:
compute_query = """
SELECT 
  SAFE_DIVIDE(SUM(arrival_delay * departure_delay), SUM(departure_delay * departure_delay)) AS alpha
FROM
(
  SELECT 
    RAND() AS split_rand,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights` 
  WHERE
    departure_airport = 'DEN' AND arrival_airport = 'LAX'
)
WHERE
  split_rand < 0.8
"""

In [19]:
import google.datalab.bigquery as bq

results = bq.Query(compute_query).execute().result().to_dataframe()
results

,alpha
0,0.976123


In [20]:
compute_rmse = """
SELECT 
  dataset,
  SQRT(AVG((arrival_delay - ALPHA * departure_delay) * (arrival_delay - ALPHA * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM
(
  SELECT
    IF (RAND() < 0.8, 'train', 'eval') AS dataset,
    departure_delay,
    arrival_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    departure_airport = 'DEN' AND arrival_airport = 'LAX'
)
GROUP BY
  dataset
"""

In [21]:
alpha = results['alpha'][0]
bq.Query(compute_query.replace('ALPHA', str(alpha))).execute().result()

alpha
0.974849335746


In [27]:
compute_alpha = """
SELECT
  SAFE_DIVIDE(SUM(arrival_delay * departure_delay), SUM(departure_delay * departure_delay)) as alpha
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  arrival_airport = 'LAX' AND departure_airport = 'DEN' AND
  MOD(ABS(FARM_FINGERPRINT(date)), 10) < 8
"""

In [30]:
result = bq.Query(compute_alpha).execute().result().to_dataframe()
alpha = result['alpha'][0]

In [31]:
compute_rmse = """
SELECT
  IF (MOD(ABS(FARM_FINGERPRINT(date)), 10) < 8, 'train', 'eval') AS dataset,
  SQRT(AVG((arrival_delay - ALPHA * departure_delay) * (arrival_delay - ALPHA * departure_delay))) AS rmse,
  COUNT(arrival_delay) AS num_flights
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  arrival_airport = 'LAX' AND departure_airport = 'DEN'
GROUP BY
  dataset
"""

In [34]:
bq.Query(compute_rmse.replace('ALPHA', str(alpha))).execute().result()

dataset,rmse,num_flights
train,13.1607120015,64018
eval,12.7646852443,15671
